# Initialization. Load previous state. Load modules

In [3]:
#jupyter nbconvert --to python 
#to run as script

import dill
# Restore the entire session
dill.load_session('PTRMS_reg_all.db')

ModuleNotFoundError: No module named 'tabpfn'

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#from tabpfn import TabPFNRegressor
#from xgboost import XGBRegressor
#from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.svm import SVR


# Load datasets froms .scv

In [2]:
# Load the GUM data
tabla = pd.read_csv("mezclas.csv", header=0, decimal=',')  # Assuming the first row contains headers

# Extract the relevant columns
Xm = tabla.iloc[:, 9:]
# Calculate row sums
suma = Xm.sum(axis=1)
# Normalize the data
XmN = Xm.div(suma, axis=0)

gum_ID_b=tabla.iloc[:, 1]
gum_ID_s=tabla.iloc[:, 0]

gum_X=XmN

gum1_y=tabla.iloc[:, 3]
gum2_y=tabla.iloc[:, 4]
gum3_y=tabla.iloc[:, 5]

In [3]:
# Load the NOC data - already normalized
tabla = pd.read_csv("Noc_140_reg.csv", header=0, decimal='.',sep=",")  # Assuming the first row contains headers

# Extract the relevant columns 
noc_ID_o=tabla.iloc[:, 2]
noc_ID_s=tabla.iloc[:, 1]

noc_X=tabla.iloc[:, 4:]

noc_y=tabla.iloc[:, 3]


In [4]:
# Load the NOC_3T data - already normalized
tabla = pd.read_csv("Noc_3T_reg.csv", header=0, decimal='.',sep=",")  # Assuming the first row contains headers

# Extract the relevant columns
n3T_ID=tabla.iloc[:, 1]

n3T_X =tabla.iloc[:, 3:]

n3T_y =tabla.iloc[:, 2]


# Function for train and predict

In [5]:
def predict_reg_by_label(IDs, y, X, regressor, param_grid={}, cv=5, verbose=True):
    """
    Calculates predictions for test sets, each with a unique label in IDs, and selects the best parameters using GridSearchCV.

    Args:
        IDs: A pandas Series containing labels.
        y: A pandas Series or numpy array containing target labels.
        X: A pandas DataFrame or numpy array containing features.
        regressor: A scikit-learn regressor class.
        param_grid: A dictionary specifying parameter options for GridSearchCV.
        cv: Number of folds for cross-validation.
        verbose: If True, prints progress and metrics.

    Returns:
        predictions: Predictions for y as test sets.
        arv.
        best_params_per_label: Dictionary with the best parameters for each label.
    """
    unique_labels = IDs.unique()
    predictions = np.zeros_like(y, dtype=float)
    best_params_per_label = {}
    
    # Ensure X and y are numpy arrays for compatibility
    if not isinstance(X, np.ndarray):
        X = X.to_numpy()
    if not isinstance(y, np.ndarray):
        y = y.to_numpy()

    for label in unique_labels:
        mask = (IDs == label)
        X_train, X_test = X[~mask], X[mask]
        y_train, y_test = y[~mask], y[mask]

        # Use GridSearchCV to find the best parameters
        if cv>1:
            grid_search = GridSearchCV(
                estimator=regressor(),
                param_grid=param_grid,
                scoring='neg_mean_squared_error',
                cv=cv,
                verbose=verbose
            )
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_

            # Store the best parameters
            best_params_per_label[label] = grid_search.best_params_

        else:
            best_model = regressor(**param_grid)
            best_model.fit(X_train, y_train)
            best_params_per_label[label] = param_grid

        # Predict on the test set
        predictions[mask] = best_model.predict(X_test)

        if verbose:
            print(f"Label: {label}, Best Params: {best_params_per_label[label]}")
            print(f"Test Set MSE for label {label}: {round(mean_squared_error(y_test, predictions[mask]), 4)}")

    # Final metrics
    final_mse = mean_squared_error(y, predictions)
    final_arv = final_mse / np.var(y)
    
    if verbose:
        print(f"\nOverall Mean Squared Error: {round(final_mse, 4)}")
        print(f"Overall Adjusted Residual Variance (ARV): {round(final_arv, 4)}")

    return predictions, final_arv, best_params_per_label


# Train methods

## Gum 1 batch

In [6]:
#Gum1b
IDs = gum_ID_b
y = gum1_y
X=gum_X

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 1, Best Params: {'C': 0.1, 'degree': 3, 'epsilon': np.float64(0.026061815797179803), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 1: 1.5474
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 2, Best Params: {'C': 0.1, 'degree': 3, 'epsilon': np.float64(0.26061815797179805), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 2: 0.7682
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 3, Best Params: {'C': 1.0, 'degree': 3, 'epsilon': np.float64(1.0424726318871922), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 3: 0.763

Overall Mean Squared Error: 1.0233
Overall Adjusted Residual Variance (ARV): 0.1507
0.15065121443838483


In [14]:
#save results
predict_SVM_gum1b = predict_SVM

In [15]:
from sklearn.metrics import r2_score
predict_SVM_gum1b[1]=(1-r2_score(gum1_y,predict_SVM_gum1b[0]))

0.1506512144383848


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.248e+00, tolerance: 9.310e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.426e+00, tolerance: 8.701e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.368e+00, tolerance: 1.120e

0.13552947715693814
0.13370108229790975
0.24360836749128573


In [8]:
#save results
predict_LASSO_gum1b = predict_LASSO
predict_RIDGE_gum1b = predict_RIDGE
predict_LARS_gum1b  = predict_LARS

In [65]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


NameError: name 'TabPFNRegressor' is not defined

In [48]:
#save results
predict_tabPFN_gum1b = predict_tabPFN

In [63]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])


Label: 1, Best Params: {'n_estimators': 1000, 'max_features': 'sqrt'}
Test Set MSE for label 1: 1.8673
Label: 2, Best Params: {'n_estimators': 1000, 'max_features': 'sqrt'}
Test Set MSE for label 2: 2.4542
Label: 3, Best Params: {'n_estimators': 1000, 'max_features': 'sqrt'}
Test Set MSE for label 3: 0.9739

Overall Mean Squared Error: 1.7563
Overall Adjusted Residual Variance (ARV): 0.6739
0.6738786956087627


In [50]:
#save results
predict_RF_gum1b = predict_RF

In [56]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits


/home/granitto/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:24:26] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Label: 1, Best Params: {'colsample_bytree': 0.75, 'device': 'cuda', 'eta': 0.25, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label 1: 3.7693
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 2, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.1, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label 2: 1.6308
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 3, Best Params: {'colsample_bytree': 0.75, 'device': 'cuda', 'eta': 0.25, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label 3: 1.1946

Overall Mean Squared Error: 2.187
Overall Adjusted Residual Variance (ARV): 0.8391
0.8391485529388174


In [57]:
#save results
predict_XGB_gum1b = predict_XGB

In [ ]:
dill.dump_session('PTRMS_reg_all.db')

## Gum 2 batch

In [9]:
#Gum2b
IDs = gum_ID_b
y = gum2_y
X=gum_X

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 1, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.6307514017712957), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 1: 2.7635
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 2, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.15768785044282393), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 2: 1.6384
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 3, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(1.2615028035425915), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 3: 1.97

Overall Mean Squared Error: 2.1223
Overall Adjusted Residual Variance (ARV): 0.6729
0.672932555636145


In [35]:
#save results
predict_SVM_gum2b = predict_SVM

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.185e+00, tolerance: 1.395e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.417e+00, tolerance: 1.330e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.433e+00, tolerance: 1.542e

0.366152904401814
0.23413082219482165
0.2977643193703833


In [11]:
#save results
predict_LASSO_gum2b = predict_LASSO
predict_RIDGE_gum2b = predict_RIDGE
predict_LARS_gum2b  = predict_LARS

In [61]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


0.7054944467774973


In [62]:
#save results
predict_tabPFN_gum2b = predict_tabPFN

In [63]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])


0.9082446703822412


In [64]:
#save results
predict_RF_gum2b = predict_RF

In [65]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 1, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.05, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label 1: 2.6779
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 2, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.05, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label 2: 2.0017
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 3, Best Params: {'colsample_bytree': 1, 'device': 'cuda', 'eta': 0.05, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.8}
Test Set MSE for label 3: 2.5928

Overall Mean Squared Error: 2.426
Overall Adjusted Residual Variance (ARV): 0.7692
0.7692440011583352


In [66]:
#save results
predict_XGB_gum2b = predict_XGB

In [ ]:
dill.dump_session('PTRMS_reg_all.db')

## Gum 3 batch

In [12]:
#Gum3b
IDs = gum_ID_b
y = gum3_y
X=gum_X

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 1, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.24134238345841436), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 1: 3.3401
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 2, Best Params: {'C': 0.01, 'degree': 3, 'epsilon': np.float64(0.4826847669168287), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 2: 2.5403
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 3, Best Params: {'C': 0.01, 'degree': 3, 'epsilon': np.float64(0.4826847669168287), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 3: 4.0116

Overall Mean Squared Error: 3.3053
Overall Adjusted Residual Variance (ARV): 1.3696
1.3695627892176458


In [38]:
#save results
predict_SVM_gum3b = predict_SVM

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.190e+00, tolerance: 8.948e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.680e+00, tolerance: 9.325e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.280e+00, tolerance: 9.385e

0.5104467217943559
0.4567954371322042
0.6468629345933185


In [14]:
#save results
predict_LASSO_gum3b = predict_LASSO
predict_RIDGE_gum3b = predict_RIDGE
predict_LARS_gum3b  = predict_LARS

In [70]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1


0.9478631520852725


In [71]:
#save results
predict_tabPFN_gum3b = predict_tabPFN

In [72]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])


1.152932438543176


In [73]:
#save results
predict_RF_gum3b = predict_RF

In [74]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 1, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.25, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label 1: 4.1626
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 2, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.1, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.8}
Test Set MSE for label 2: 2.5213
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 3, Best Params: {'colsample_bytree': 0.75, 'device': 'cuda', 'eta': 0.25, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label 3: 1.9921

Overall Mean Squared Error: 2.8819
Overall Adjusted Residual Variance (ARV): 1.1941
1.1941040977802293


In [75]:
#save results
predict_XGB_gum3b = predict_XGB

In [ ]:
dill.dump_session('PTRMS_reg_all.db')

## Nocciole sample

In [15]:
#Noc S
IDs=noc_ID_s
y = noc_y
X = noc_X

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: AKt1, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.6242182611074432), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label AKt1: 2.2663
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: AKt2, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.6242182611074432), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label AKt2: 2.9869
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: AKt3, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.6242182611074432), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label AKt3: 1.139
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: AKt4, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.6242182611074432), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label AKt4: 1.5166
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: AKt5, Best P

In [41]:
#save results
predict_SVM_nocs = predict_SVM

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.193e-01, tolerance: 2.215e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.467e-01, tolerance: 1.555e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.910e-01, tolerance: 2.234e

0.06306079929568115
0.06950690576850284
0.21893057484911757


In [17]:
#save results
predict_LASSO_nocs = predict_LASSO
predict_RIDGE_nocs = predict_RIDGE
predict_LARS_nocs  = predict_LARS

In [82]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


0.31790534420650707


In [83]:
#save results
predict_tabPFN_nocs = predict_tabPFN

In [84]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])


0.9934521895653889


In [85]:
#save results
predict_RF_nocs = predict_RF

In [86]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: AKt1, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.25, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label AKt1: 3.9999
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: AKt2, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.05, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label AKt2: 10.3638
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: AKt3, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.05, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label AKt3: 11.2529
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: AKt4, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.1, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label AKt4: 1.1229
Fitting 5 folds for each of 81 candidates, totallin

In [87]:
#save results
predict_XGB_nocs = predict_XGB

In [ ]:
dill.dump_session('PTRMS_reg_all.db')

## Nocciole origin

In [18]:
#Noc o
IDs=noc_ID_o
y = noc_y
X = noc_X

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: AK, Best Params: {'C': 0.01, 'degree': 3, 'epsilon': np.float64(0.6242182611074432), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label AK: 18.6468
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: RO, Best Params: {'C': 0.01, 'degree': 3, 'epsilon': np.float64(0.3121091305537216), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label RO: 3.2347
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: TGTP, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.6242182611074432), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label TGTP: 9.3155

Overall Mean Squared Error: 10.399
Overall Adjusted Residual Variance (ARV): 1.6659
1.665920308368327


In [44]:
#save results
predict_SVM_noco = predict_SVM

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.644e-01, tolerance: 1.269e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.331e-01, tolerance: 1.705e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.672e-01, tolerance: 1.109e

0.20619882862161734
0.21742389747185617
0.6226394904409183


In [20]:
#save results
predict_LASSO_noco = predict_LASSO
predict_RIDGE_noco = predict_RIDGE
predict_LARS_noco  = predict_LARS

In [91]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


1.5933016218700624


In [92]:
#save results
predict_tabPFN_noco = predict_tabPFN

In [93]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])


1.7513532420658415


In [94]:
#save results
predict_RF_noco = predict_RF

In [95]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: AK, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.1, 'max_depth': 5, 'n_estimators': 1000, 'subsample': 0.8}
Test Set MSE for label AK: 19.2649
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: RO, Best Params: {'colsample_bytree': 1, 'device': 'cuda', 'eta': 0.1, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.8}
Test Set MSE for label RO: 3.4153
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: TGTP, Best Params: {'colsample_bytree': 0.75, 'device': 'cuda', 'eta': 0.25, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label TGTP: 16.6895

Overall Mean Squared Error: 13.1232
Overall Adjusted Residual Variance (ARV): 2.1023
2.102345516273577


In [96]:
#save results
predict_XGB_noco = predict_XGB

In [ ]:
dill.dump_session('PTRMS_reg_all.db')

## Nocciole 3 temperatures

In [21]:
#Noc 3T
IDs=n3T_ID
y = n3T_y
X = n3T_X

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: TGTP_125_t3, Best Params: {'C': 1.0, 'degree': 3, 'epsilon': np.float64(0.1460040667241841), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label TGTP_125_t3: 39.13
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: TGTP_125_t5, Best Params: {'C': 1.0, 'degree': 3, 'epsilon': np.float64(0.1460040667241841), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label TGTP_125_t5: 162.9869
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: TGTP_140_t5, Best Params: {'C': 1.0, 'degree': 3, 'epsilon': np.float64(0.1460040667241841), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label TGTP_140_t5: 5.6821
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: TGTP_140_t4, Best Params: {'C': 1.0, 'degree': 3, 'epsilon': np.float64(0.1460040667241841), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label TGTP_140_t4: 16.0086
Fitting 5 folds for each o

In [47]:
#save results
predict_SVM_n3T = predict_SVM

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.929e+00, tolerance: 8.788e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.901e+00, tolerance: 1.115e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.503e+00, tolerance: 1.092e

0.20053017250067678
0.2010959636653675
0.6019264680610882


In [23]:
#save results
predict_LASSO_n3T = predict_LASSO
predict_RIDGE_n3T = predict_RIDGE
predict_LARS_n3T  = predict_LARS

In [100]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.1

/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/home/granitto/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3416: RuntimeWarning: overflow encountered in power
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1


1.2278000232916793


In [101]:
#save results
predict_tabPFN_n3T = predict_tabPFN

In [102]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])



9.682063262916738


In [103]:
#save results
predict_RF_n3T = predict_RF

In [104]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: TGTP_125_t3, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.05, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label TGTP_125_t3: 183.4396
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: TGTP_125_t5, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.25, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label TGTP_125_t5: 233.1643
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: TGTP_140_t5, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.05, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label TGTP_140_t5: 12.0324
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: TGTP_140_t4, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.25, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.8}
Test Set MSE for label TGTP_140_t4: 24.2

In [105]:
#save results
predict_XGB_n3T = predict_XGB

In [51]:
dill.dump_session('PTRMS_reg_all.db')

NameError: name 'dill' is not defined

## Gum 1 sample 

In [24]:
#Gum1s
IDs = gum_ID_s
y = gum1_y
X=gum_X

In [49]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: GB_14, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.5212363159435961), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label GB_14: 1.3137
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: GB_12, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.026061815797179803), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label GB_12: 1.8034
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: GB_21, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.5212363159435961), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label GB_21: 4.9117
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: GB_8, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.026061815797179803), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label GB_8: 0.1127
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: G

In [50]:
#save results
predict_SVM_gum1s = predict_SVM

In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.705e+00, tolerance: 1.463e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.665e+00, tolerance: 1.315e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.397e+00, tolerance: 1.484e

0.19962377456195182
0.2204107913566857
0.2876494282036435


In [26]:
#save results
predict_LASSO_gum1s = predict_LASSO
predict_RIDGE_gum1s = predict_RIDGE
predict_LARS_gum1s  = predict_LARS

In [46]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


(array([ 2.83651170e-04,  1.56391121e-04,  1.35355105e-04,  3.61981106e+00,
        3.53651786e+00,  3.61045694e+00,  1.56238396e-02,  3.04278601e-02,
        1.03122070e-02, -1.75617868e-04, -1.37636554e-04, -5.09526581e-05,
       -4.09356086e-04, -3.12448363e-04, -7.10453896e-04, -2.24138028e-04,
       -2.17172434e-04, -3.45015316e-04, -8.93404125e-04,  6.27074158e-04,
       -4.80045506e-04, -5.14606363e-04, -7.08503881e-04, -3.99269513e-04,
       -4.31744847e-05, -4.63379547e-06,  1.45861995e-04,  8.46214825e-05,
       -2.64480972e-04, -2.72881851e-04, -2.38035922e-04,  3.66985106e+00,
        3.50843215e+00,  3.31311202e+00,  3.36717606e+00,  7.25888740e-03,
        1.14073406e-03,  3.26497667e-03,  6.71430782e-04,  7.99434492e-05,
        7.39404350e-04,  3.42811516e-04,  1.83998636e-04,  3.38581085e-05,
       -1.30705506e-04, -1.41215351e-04,  7.42441625e-06,  8.30213928e+00,
        8.30859947e+00,  8.25074291e+00,  8.36183739e+00,  6.33775053e-05,
        2.28628109e-04, 

In [48]:
#save results
predict_tabPFN_gum1s = predict_tabPFN

In [49]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])


0.6788577109387715


In [50]:
#save results
predict_RF_gum1s = predict_RF

In [56]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits


/home/granitto/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:24:26] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Label: 1, Best Params: {'colsample_bytree': 0.75, 'device': 'cuda', 'eta': 0.25, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label 1: 3.7693
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 2, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.1, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label 2: 1.6308
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 3, Best Params: {'colsample_bytree': 0.75, 'device': 'cuda', 'eta': 0.25, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label 3: 1.1946

Overall Mean Squared Error: 2.187
Overall Adjusted Residual Variance (ARV): 0.8391
0.8391485529388174


In [57]:
#save results
predict_XGB_gum1s = predict_XGB

In [ ]:
dill.dump_session('PTRMS_reg_all.db')

## Gum 2 sample

In [27]:
#Gum2b
IDs = gum_ID_s
y = gum2_y
X=gum_X

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 1, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.6307514017712957), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 1: 2.7635
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 2, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(0.15768785044282393), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 2: 1.6384
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: 3, Best Params: {'C': 0.05, 'degree': 3, 'epsilon': np.float64(1.2615028035425915), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label 3: 1.97

Overall Mean Squared Error: 2.1223
Overall Adjusted Residual Variance (ARV): 0.6729
0.672932555636145


In [35]:
#save results
predict_SVM_gum2s = predict_SVM

In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.137e+00, tolerance: 2.251e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.484e+00, tolerance: 1.945e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.393e+01, tolerance: 2.314e

0.4313402736821018
0.48315699770225545
0.4380608312457752


In [29]:
#save results
predict_LASSO_gum2s = predict_LASSO
predict_RIDGE_gum2s = predict_RIDGE
predict_LARS_gum2s  = predict_LARS

In [61]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


0.7054944467774973


In [62]:
#save results
predict_tabPFN_gum2s = predict_tabPFN

In [63]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])


0.9082446703822412


In [64]:
#save results
predict_RF_gum2s = predict_RF

In [65]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 1, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.05, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label 1: 2.6779
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 2, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.05, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.6}
Test Set MSE for label 2: 2.0017
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 3, Best Params: {'colsample_bytree': 1, 'device': 'cuda', 'eta': 0.05, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.8}
Test Set MSE for label 3: 2.5928

Overall Mean Squared Error: 2.426
Overall Adjusted Residual Variance (ARV): 0.7692
0.7692440011583352


In [66]:
#save results
predict_XGB_gum2s = predict_XGB

In [ ]:
dill.dump_session('PTRMS_reg_all.db')

## Gum 3 sample

In [32]:
#Gum3b
IDs = gum_ID_s
y = gum3_y
X=gum_X

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#SVM-eps
param_grid = {
    'kernel':['linear'],
    'epsilon':[np.std(y) * v for v in [0.01,0.05,0.1,0.2,0.4]],
    'C':[0.01,0.05,0.1,1.0,10],
    'degree':[3],
    'gamma':['scale']
}
# Estandarización
predict_SVM  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=SVR, param_grid = param_grid,verbose=True)
print(predict_SVM[1])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: GB_14, Best Params: {'C': 0.01, 'degree': 3, 'epsilon': np.float64(0.12067119172920718), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label GB_14: 0.2072
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Label: GB_12, Best Params: {'C': 0.01, 'degree': 3, 'epsilon': np.float64(0.12067119172920718), 'gamma': 'scale', 'kernel': 'linear'}
Test Set MSE for label GB_12: 0.5562
Fitting 5 folds for each of 25 candidates, totalling 125 fits


KeyboardInterrupt: 

In [35]:
#save results
predict_SVM_gum3s = predict_SVM

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#linear models
param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_LASSO  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lasso, param_grid = param_grid,verbose=False)
print(predict_LASSO[1])

param_grid = {
    'alpha': [0.001,0.01,0.1,1,10],
    'random_state': [42]
}
predict_RIDGE  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Ridge, param_grid = param_grid,verbose=False)
print(predict_RIDGE[1])

param_grid = {
    'n_nonzero_coefs': [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20],
    'random_state': [42]
}
predict_LARS  = predict_reg_by_label(IDs=IDs, y=y, X=scaler.fit_transform(X),regressor=linear_model.Lars, param_grid = param_grid,verbose=False)
print(predict_LARS[1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.460e+00, tolerance: 1.384e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.750e+00, tolerance: 1.365e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.288e+01, tolerance: 1.407e

0.8567950479823386
1.0512276294011333
0.7617988044444701


In [35]:
#save results
predict_LASSO_gum3s = predict_LASSO
predict_RIDGE_gum3s = predict_RIDGE
predict_LARS_gum3s  = predict_LARS

In [ ]:
#tabPFN
param_grid = {
    'ignore_pretraining_limits':True
}
predict_tabPFN  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=TabPFNRegressor, param_grid = param_grid,verbose=False)
print(predict_tabPFN[1])


In [71]:
#save results
predict_tabPFN_gum3s = predict_tabPFN

In [72]:
#RF
param_grid = {
    'n_estimators': 1000,
    'max_features':'sqrt'
}
predict_RF  = predict_reg_by_label(cv=0,IDs=IDs, y=y, X=X,regressor=RandomForestRegressor, param_grid = param_grid,verbose=True)
print(predict_RF[1])


1.152932438543176


In [73]:
#save results
predict_RF_gum3s = predict_RF

In [74]:
#XGBoost
param_grid = {
    'device':['cpu'],
    'n_estimators': [1000], 
    'eta':[0.25,0.10,0.05], 
    'max_depth':[1,3,5], 
    'subsample':[1,0.8,0.6], 
    'colsample_bytree':[1,0.75,0.5]
}
predict_XGB = predict_reg_by_label(IDs=IDs, y=y, X=X,regressor=XGBRegressor, param_grid = param_grid,verbose=True)
print(predict_XGB[1])

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 1, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.25, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label 1: 4.1626
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 2, Best Params: {'colsample_bytree': 0.5, 'device': 'cuda', 'eta': 0.1, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 0.8}
Test Set MSE for label 2: 2.5213
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Label: 3, Best Params: {'colsample_bytree': 0.75, 'device': 'cuda', 'eta': 0.25, 'max_depth': 1, 'n_estimators': 1000, 'subsample': 1}
Test Set MSE for label 3: 1.9921

Overall Mean Squared Error: 2.8819
Overall Adjusted Residual Variance (ARV): 1.1941
1.1941040977802293


In [75]:
#save results
predict_XGB_gum3s = predict_XGB

In [ ]:
dill.dump_session('PTRMS_reg_all.db')

In [3]:
print("Data ","  RF  ","TabPFN","  LASSO ","  SVM  ","  XGB  ")
print("Gum1_b ",round(predict_RF_gum1b[1],4),round(predict_TabPFN_gum1b[1],4),round(predict_LASSO_gum1b[1],4),round(predict_SVM_gum1b[1],4),round(predict_XGB_gum1b[1],4))
print("Gum2_b ",round(predict_RF_gum2b[1],4),round(predict_TabPFN_gum2b[1],4),round(predict_LASSO_gum2b[1],4),round(predict_SVM_gum2b[1],4),round(predict_XGB_gum2b[1],4))
print("Gum3_b ",round(predict_RF_gum3b[1],4),round(predict_TabPFN_gum3b[1],4),round(predict_LASSO_gum3b[1],4),round(predict_SVM_gum3b[1],4),round(predict_XGB_gum3b[1],4))
print("Gum1_s ",round(predict_RF_gum1s[1],4),round(predict_TabPFN_gum1s[1],4),round(predict_LASSO_gum1s[1],4),round(predict_SVM_gum1s[1],4),round(predict_XGB_gum1s[1],4))
print("Gum2_s ",round(predict_RF_gum2s[1],4),round(predict_TabPFN_gum2s[1],4),round(predict_LASSO_gum2s[1],4),round(predict_SVM_gum2s[1],4),round(predict_XGB_gum2s[1],4))
print("Gum3_s ",round(predict_RF_gum3s[1],4),round(predict_TabPFN_gum3s[1],4),round(predict_LASSO_gum3s[1],4),round(predict_SVM_gum3s[1],4),round(predict_XGB_gum3s[1],4))
print("Noc_s  ",round(predict_RF_nocs[1],4) ,round(predict_TabPFN_nocs[1],4) ,round(predict_LASSO_nocs[1],4) ,round(predict_SVM_nocs[1],4) ,round(predict_XGB_nocs[1],4))
print("Noc_o  ",round(predict_RF_noco[1],4) ,round(predict_TabPFN_noco[1],4) ,round(predict_LASSO_noco[1],4) ,round(predict_SVM_noco[1],4) ,round(predict_XGB_noco[1],4))
print("Noc_3T ",round(predict_RF_n3T[1],4)  ,round(predict_TabPFN_n3T[1],4)  ,round(predict_LASSO_n3T[1],4)  ,round(predict_SVM_n3T[1],4)  ,round(predict_XGB_n3T[1],4))

Data    RF   TabPFN   LASSO    SVM     XGB  


NameError: name 'predict_RF_gum1b' is not defined

In [36]:
print("Data   ", "  LASSO  ")
print("Gum1_b ", round(predict_LASSO_gum1b[1], 4))
print("Gum2_b ", round(predict_LASSO_gum2b[1], 4))
print("Gum3_b ", round(predict_LASSO_gum3b[1], 4))
print("Gum1_s ", round(predict_LASSO_gum1s[1], 4))
print("Gum2_s ", round(predict_LASSO_gum2s[1], 4))
print("Gum3_s ", round(predict_LASSO_gum3s[1], 4))
print("Noc_s  ", round(predict_LASSO_nocs[1], 4))
print("Noc_o  ", round(predict_LASSO_noco[1], 4))
print("Noc_3T ", round(predict_LASSO_n3T[1], 4))

Data      LASSO  
Gum1_b  0.1355
Gum2_b  0.3662
Gum3_b  0.5104
Gum1_s  0.1996
Gum2_s  0.4313
Gum3_s  0.8568
Noc_s   0.0631
Noc_o   0.2062
Noc_3T  0.2005
